In [2]:
## 1. Setup


In [ ]:
#import all the required libraries
import matplotlib.pyplot as plt
import numpy as np
import os,cv2,random,shutil,keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras import layers, models

import pandas as pd
from PIL import Image as im

In [ ]:
base_dir_data = "C:/Users/rarvr/Downloads/archive2/chest_xray"
training_main="C:/Users/rarvr/Downloads/archive2/chest_xray/train"
testing_main="C:/Users/rarvr/Downloads/archive2/chest_xray/test"
val_main="C:/Users/rarvr/Downloads/archive2/chest_xray/val"
train_norm="C:/Users/rarvr/Downloads/archive2/chest_xray/train/Normal"

os.mkdir(base_dir_data+'/processing')
os.mkdir(training_main+'/processing')
os.mkdir(val_main+'/processing')
os.mkdir(testing_main+'/processing')


In [ ]:
train_dir = os.path.join('C:/Users/rarvr/Downloads/data/sorted_xray/train')
validation_dir = os.path.join('C:/Users/rarvr/Downloads/data/sorted_xray/val')
test_dir = os.path.join('C:/Users/rarvr/Downloads/data/sorted_xray/test')

val_dir_norm=val_main+"/NORMAL"

val_dir_pneum=val_main+"/PNEUMONIA"

test_dir_norm=testing_main+"/NORMAL"
test_dir_pneum=testing_main+"/PNEUMONIA"

train_dir_norm=training_main+"/NORMAL"
train_dir_pneum=training_main+"/PNEUMONIA"
os.mkdir(val_main+"/NORMAL")
os.mkdir(val_main+"/PNEUMONIA")
os.mkdir(testing_main+"/NORMAL")
os.mkdir(testing_main+"/PNEUMONIA")
os.mkdir(training_main+"/NORMAL")
os.mkdir(training_main+"/PNEUMONIA")
len(os.listdir(train_dir_pneum))


In [ ]:
img_sze = (224, 224)
batch_size=60
label_mode="binary"

training_data_resized = tf.keras.preprocessing.image_dataset_from_directory(directory=training_main, 
                                                                 image_size=img_sze,
                                                                 label_mode=label_mode, 
                                                                 batch_size=batch_size,
                                                                 shuffle=1
                                                                )

testing_data_resized = tf.keras.preprocessing.image_dataset_from_directory(directory=testing_main,
                                                                image_size=img_sze,
                                                                batch_size=batch_size,
                                                                label_mode=label_mode,shuffle=0)


validation_data_resized = tf.keras.preprocessing.image_dataset_from_directory(directory=val_main,
                                                                image_size=img_sze,
                                                                batch_size=batch_size,
                                                                label_mode=label_mode, shuffle=0)

In [ ]:
#resizing all the datasets training,testing and validation

trained_data = training_data_resized.prefetch(buffer_size=tf.data.AUTOTUNE)
test_data = testing_data_resized.prefetch(buffer_size=tf.data.AUTOTUNE)
val_data = validation_data_resized.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
## modelling applied on the CNN model of Cov2D altering and maxpoolong also applied
model = models.Sequential().add(layers.Conv2D(8, (3, 3), activation='relu',input_shape=(224, 224, 3)))

model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(126, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
# the final layer in the model is flattened and dropedout
model.add(layers.Flatten())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(256, activation='relu'))
#the coutput layer includes the sigmoid function as an activator
model.add(layers.Dense(1, activation='sigmoid'))
#the description of the models layers are printed
model.summary()

In [ ]:
#optimization of the model using keras optimizer (adam)

model.compile(loss='binary_crossentropy',
             optimizer=tf.keras.optimizers.Adam(),
             metrics=["acc", tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.SpecificityAtSensitivity(0.6), tf.keras.metrics.SensitivityAtSpecificity(0.4)])


In [ ]:
# training of the model's preprocessed data
#when using epochs more then 10 then the precision decreases as a result of overfitting
epochs=5
history = model.fit(trained_data,
                   epochs=epochs,
                   steps_per_epoch=len(trained_data),
                   validation_data=val_data,
                   validation_steps=len(val_data),
                   verbose=1)

In [ ]:
# finding the performace of the model
acc = history.history['acc']
val_acc = history.history['val_acc']
print(list(val_acc))
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, flt(len(acc)) + 1)

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'yellow', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend().figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'yellow', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()